* U.S. Manufactured Goods: Domestic Production vs Imports (2013–2024)

In [2]:
# process price index
import pandas as pd

df_ppi = pd.read_csv("../data/Processed/price_index/producer_price_index.csv", parse_dates=['YearMonth'])
df_ipi = pd.read_csv("../data/Processed/price_index/import_price_index.csv", parse_dates=['YearMonth'])

# convert to quarter data
df_ipi_q = df_ipi.set_index('YearMonth')['import_price_index'].resample('Q').mean().reset_index(name='ImportPI')
df_ppi_q = df_ppi.set_index('YearMonth')['producer_price_index'].resample('Q').mean().reset_index(name='ProducerPI')
df_merge = pd.merge(df_ipi_q, df_ppi_q, how = "outer", on = "YearMonth")

# compute price difference(import - domestic)
df_merge['PriceDiff'] = df_merge['ImportPI'] - df_merge['ProducerPI']

# rename for easy merge
df_merge = df_merge.rename(columns = {'YearMonth': 'Date'})
df_merge['Date'] = (pd.PeriodIndex(df_merge['Date'], freq='Q').to_timestamp(how='start'))

print(df_merge.tail())

         Date    ImportPI  ProducerPI   PriceDiff
44 2024-01-01  121.900000  247.289000 -125.389000
45 2024-04-01  122.933333  249.286333 -126.353000
46 2024-07-01  122.933333  248.451333 -125.518000
47 2024-10-01  123.166667  246.654333 -123.487667
48 2025-01-01         NaN  249.438000         NaN


In [3]:
# policy dataframe
import pandas as pd

policies = pd.DataFrame([
    {'start':'2015-01-01', 'end':'2016-06-30', 'policy':'A'},
    {'start':'2016-07-01', 'end':'2018-03-31', 'policy':'B'},
    {'start':'2018-04-01', 'end':'2020-12-31', 'policy':'C'},
])
policies['start'] = pd.to_datetime(policies['start'])
policies['end']   = pd.to_datetime(policies['end'])

In [4]:
import pandas as pd
import altair as alt

# prepare dataframe
df_imp = pd.read_csv('../data/Processed/import_domestic/import.csv', names=['Quarter','Import_Value'],skiprows=1)
df_dmst  = pd.read_csv('../data/Processed/import_domestic/domestic_gross_output.csv', names=['Quarter','Domestic_Gross_Output_Value'],skiprows=1)

df = df_imp.merge(df_dmst, on='Quarter')
df['Date'] = pd.PeriodIndex(df['Quarter'], freq='Q').to_timestamp()
df['Share']   = df['Import_Value'] / (df['Import_Value'] + df['Domestic_Gross_Output_Value']) * 100
df = pd.merge(df, df_merge, how = "left", on = 'Date')

df_bar = df.melt(
    id_vars=['Date'],
    value_vars=['Domestic_Gross_Output_Value','Import_Value'],
    var_name='Type',
    value_name='Amount'
)

# build selector
nearest = alt.selection_point(
    fields=['Date'],
    nearest=True,
    on='mouseover',
    empty='none'
)

# build policy row
policy_bars = alt.Chart(policies).mark_rect().encode(
    x='start:T',
    x2='end:T',
    y=alt.value(0),        
    y2=alt.value(1),
    color=alt.Color('policy:N',
                    scale=alt.Scale(scheme='category10'))
).properties(height=30)

policy_labels = alt.Chart(policies).mark_text(
    baseline='middle', dy=-10, color='black'
).encode(
    x=alt.X('start:T', title=None),   
    text='policy:N'
).properties(height=30)

policy_row = alt.layer(policy_bars, policy_labels).properties(
    width=800,   
    title='Policy'
)

# build price index difference line chart
diff_line = alt.Chart(df).mark_line(color='steelblue', strokeWidth=2).encode(
    x=alt.X('Date:T', title='Date'),
    y=alt.Y('PriceDiff:Q', title='IPI - PPI', scale=alt.Scale(domain=[-140, -60])),
    tooltip=['Date:T', 'PriceDiff:Q']
)

points = alt.Chart(df).mark_point(
    color='steelblue', size=50
).encode(
    x='Date:T',
    y='PriceDiff:Q'
)

trend_line = diff_line.transform_regression(
    'Date', 'PriceDiff', method='poly', order=3
).mark_line(color='red', strokeDash=[4,2], strokeWidth=2)

price_text = alt.Chart(df).transform_filter(nearest).mark_text(
    dy=-10, color='black'
).encode(
    x='Date:T',
    y='PriceDiff:Q',
    text=alt.Text('PriceDiff:Q', format='.2f'))

line_rule = alt.Chart(df).mark_rule(color='gray').encode(
    x='Date:T'
).transform_filter(nearest)

price_index_chart = alt.layer(diff_line, points, trend_line, line_rule, price_text).properties(
    width=800,   
    title='Price Index(Normalizaed)'
)

# build top
bars = alt.Chart(df_bar).mark_bar().encode(
    x=alt.X('Date:T',),
    y=alt.Y('Amount:Q', title='Absolute Value (Billion $)'),
    color=alt.Color('Type:N',
                    scale=alt.Scale(domain=['Domestic_Gross_Output_Value','Import_Value'],
                                    range=['#4C78A8','#F58518']),
                    legend=alt.Legend(title='Category'))
)

bars_points = alt.Chart(df_bar).mark_point(size=100, opacity=0).encode(
    x='Date:T',
    y='Amount:Q'
).add_selection(nearest)

bars_rule = alt.Chart(df).mark_rule(color='gray').encode(
    x='Date:T'
).transform_filter(nearest)

bars_text = alt.Chart(df_bar).transform_filter(nearest).mark_text(
    dy=-5
).encode(
    x='Date:T',
    y='Amount:Q',
    text=alt.Text('Amount:Q', format='.0f'),
    detail='Type:N'      
)

top = alt.layer(bars, bars_points, bars_rule, bars_text).properties(
    width=800, height=300,
    title='Absolute Value: Import vs Domestic'
)

# build bottom
raw_line = alt.Chart(df).mark_line(
    color='steelblue', strokeWidth=2
).encode(
    x='Date:T',
    y=alt.Y('Share:Q', title='Import Share (%)', scale = alt.Scale(domain=[6, 10])),
)

points = alt.Chart(df).mark_point(
    color='steelblue', size=50
).encode(
    x='Date:T',
    y='Share:Q'
)

trend = alt.Chart(df).transform_regression(
    'Date', 'Share', method='poly', order=3
).mark_line(
    color='red', strokeDash=[4,2], strokeWidth=2
).encode(
    x='Date:T',
    y='Share:Q'
)

line_rule = alt.Chart(df).mark_rule(color='gray').encode(
    x='Date:T'
).transform_filter(nearest)

line_text = alt.Chart(df).transform_filter(nearest).mark_text(
    dy=-10, color='black'
).encode(
    x='Date:T',
    y='Share:Q',
    text=alt.Text('Share:Q', format='.2f')
)

bottom = alt.layer(raw_line, points, trend, line_rule, line_text).properties(
    width=800, height=200,
    title='Import Share'
)

# assemble
chart = alt.vconcat(policy_row, price_index_chart, top, bottom).resolve_scale(
    x='shared',    
    y='independent' 
)

chart

/var/folders/gc/drvvdqb93dqg4y0zt5zljf5w0000gn/T/ipykernel_41920/3904470832.py:97: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  ).add_selection(nearest)


alt.VConcatChart(...)

In [31]:
import pandas as pd
import altair as alt

# Add dummy IPI - PPI difference for demo (replace with real 'PriceDiff')
df['PriceDiff'] = df_merge['PriceDiff'] 
print(df.dtypes)

# Reshape bar chart data
df_bar = df.melt(
    id_vars=['Date'],
    value_vars=['Domestic_Gross_Output_Value', 'Import_Value'],
    var_name='Type',
    value_name='Amount'
)

df_bar['Type'] = df_bar['Type'].replace({
    'Domestic_Gross_Output_Value': 'Domestic Output',
    'Import_Value': 'Import Value'   
})

# plot

# build the brush
brush = alt.selection_interval(encodings=['x'])

# line chart(ipi- ppi)
base = alt.Chart(df).encode(x=alt.X('Date:T', title="Year", axis=alt.Axis(grid=False)),
                            tooltip=[
              alt.Tooltip('Date:T', title='Year', format='%Y'),
              alt.Tooltip('Date:T', title='Quarter', timeUnit='quarter'),
              alt.Tooltip('PriceDiff:Q', title='IPI - PPI', format='.2f')])

main_line = base.mark_line(color='steelblue', strokeWidth=2).encode(
    y=alt.Y('PriceDiff:Q', title='IPI - PPI (Index, 1982 = 100)', scale=alt.Scale(domain=[-140, -60]), axis=alt.Axis(grid=False))
)

main_points = base.mark_point(color='steelblue', size=50).encode(
    y='PriceDiff:Q'
)

'''main_trend = main_line.transform_regression(
    'Date', 'PriceDiff', method='poly', order=3
).mark_line(color='red', strokeDash=[4,2], strokeWidth=2)'''

main_chart = alt.layer(main_line, main_points).add_params(
    brush
).properties(
    width=900,
    height=250,
    title='Difference Between the Manufacturing Import Price Index and the Producer Price Index'
)

# bar chart
bar_chart = alt.Chart(df_bar).transform_filter(
    brush
).mark_bar().encode(
    x=alt.X('Date:T', title="Year", axis=alt.Axis(format='%Y', grid=False, labelExpr="timeFormat(datum.value, '%m') == '01' ? timeFormat(datum.value, '%Y') : ''")),
    y=alt.Y('Amount:Q', title='Total Value of Goods (Billion $)', axis=alt.Axis(grid=False)),
    color=alt.Color('Type:N',
        scale=alt.Scale(domain=['Domestic Output', 'Import Value'],
                        range=['#4C78A8','#F58518']),
                        legend=alt.Legend(orient="top",
                                          title=None,
                                          labelFontSize=10,
                                          offset=10)),
        tooltip=[
              alt.Tooltip('Type:N', title='Type'),
              alt.Tooltip('Date:T', title='Year', format='%Y'),
              alt.Tooltip('Date:T', title='Quarter', timeUnit='quarter'),
              alt.Tooltip('Amount:Q', title='Value', format='.2f')]
        
).properties(
    width=420,
    height=200,
    title='Total Value of Domestic and Imported Manufactured Goods'
)

# Import share line chart filtered by brush
share_base = (
    alt.Chart(df)
      .transform_filter(brush)       
      .encode(
          x=alt.X('Date:T', title="Year", axis=alt.Axis(format='%Y', grid=False, labelExpr="timeFormat(datum.value, '%m') == '01' ? timeFormat(datum.value, '%Y') : ''"
                  )),
          y=alt.Y('Share:Q', title='Share of Import Goods Value (%)', scale=alt.Scale(domain=[0,10]), axis=alt.Axis(grid=False)),
          tooltip=[
              alt.Tooltip('Date:T', title='Year', format='%Y'),
              alt.Tooltip('Date:T', title='Quarter', timeUnit='quarter'),
              alt.Tooltip('Share:Q', title='Import Share (%)', format='.2f')
          ]
      )
)

line = share_base.mark_line(color='steelblue', strokeWidth=2)
points = share_base.mark_point(color='steelblue', size=20)

share_line = (
    (line + points)
      .properties(
         width=420, 
         height=200, 
         title='Share of Imported Goods Value in Manufacturing'
      )
)
# caption
caption = alt.Chart(pd.DataFrame({
    'text': ["Note: The IPI and PPI indexes are standardized to 1982=100. Values represent percentage differences relative to the 1982 base year."]
})).mark_text(
    align='left',
    fontSize=10,
    dy=10  
).encode(
    text='text'
).properties(width=850)


popup = alt.concat(bar_chart, share_line).resolve_legend(
    color='independent',
    size='independent')
final_chart = alt.vconcat(main_chart, popup, caption).properties(title = alt.TitleParams(text="Evolution of Prices, Import Scale, and Market Share in U.S. Manufacturing (2013-2024)", anchor="middle", fontSize=16)).resolve_legend(
    color='independent',
    size='independent')

final_chart

Quarter                                object
Import_Value                          float64
Domestic_Gross_Output_Value           float64
Date                           datetime64[ns]
Share                                 float64
ImportPI                              float64
ProducerPI                            float64
PriceDiff                             float64
dtype: object


alt.VConcatChart(...)

In [32]:
final_chart.save("../img/Price_Index.html")